### import time
import pandas as pd
import os
from pynput import keyboard

# 📁 Save location
save_dir = r"C:\Users\sagni\Downloads\KeyStrock Security"
os.makedirs(save_dir, exist_ok=True)

fixed_text = "secure123"
required_repeats = 5

# 👤 Get user input
username = input("Enter your name: ").strip().replace(" ", "_")
output_path = os.path.join(save_dir, f"{username}.csv")

# 🗃️ Store all sessions
all_data = []

# 🔄 State
typed_chars = []
current_keys = []
typed_count = 0

print(f"\nHello {username}!")
print(f"Please type '{fixed_text}' exactly {required_repeats} times to collect timing data.")
print("Start typing now...")

def reset_input_state():
    global typed_chars, current_keys
    typed_chars.clear()
    current_keys.clear()

def on_press(key):
    try:
        current_keys.append((key.char, 'down', time.time()))
    except AttributeError:
        pass

def on_release(key):
    global typed_count, typed_chars, current_keys, all_data

    try:
        k = key.char
    except AttributeError:
        return

    current_keys.append((k, 'up', time.time()))
    typed_chars.append(k)

    # If typed enough characters to check
    if len(typed_chars) >= len(fixed_text):
        # Check last N chars
        recent = ''.join(typed_chars[-len(fixed_text):])
        if recent == fixed_text:
            print(f"✅ Correct input detected. Saving sample {typed_count + 1}...")

            key_down_times = [x[2] for x in current_keys if x[1] == 'down'][-len(fixed_text):]
            key_up_times = [x[2] for x in current_keys if x[1] == 'up'][-len(fixed_text):]
            key_chars = [x[0] for x in current_keys if x[1] == 'down'][-len(fixed_text):]

            row = {}

            for i, ch in enumerate(key_chars):
                row[f'H.{ch}'] = round(key_up_times[i] - key_down_times[i], 5)

            for i in range(1, len(key_chars)):
                row[f'UD.{key_chars[i - 1]}.{key_chars[i]}'] = round(key_down_times[i] - key_up_times[i - 1], 5)

            all_data.append(row)
            typed_count += 1

            reset_input_state()

            if typed_count >= required_repeats:
                print("🎉 Done! Saving data...")
                df = pd.DataFrame(all_data)
                df.to_csv(output_path, index=False)
                print(f"📁 Saved dataset at: {output_path}")
                listener.stop()
        else:
            # Only reset if extra characters are typed (more than length)
            if len(typed_chars) > len(fixed_text):
                print("❌ Incorrect. Start again.")
                reset_input_state()

# ⌨️ Start keyboard listener
listener = keyboard.Listener(on_press=on_press, on_release=on_release)
listener.start()
listener.join()
